# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [2]:
# import numpy and pandas
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [3]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [4]:
# Your code here:

pokemon.head(2)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [5]:
# Your code here:

pokemon["Legendary"].value_counts()

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [6]:
# Your code here:
leg_mean = pokemon[pokemon["Legendary"]==True]["Total"].mean()
nleg_mean = pokemon[pokemon["Legendary"]==False]["Total"].mean()
leg_std = pokemon[pokemon["Legendary"]==True]["Total"].std(ddof=0)
nleg_std = pokemon[pokemon["Legendary"]==False]["Total"].std(ddof=0)
print(leg_mean)
print(nleg_mean)
print(leg_std)
print(nleg_std)

637.3846153846154
417.21360544217686
60.46682302213781
106.68776660321996


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [7]:
# Your code here:

st.ttest_ind(pokemon[pokemon["Legendary"]==True]["Total"],pokemon[pokemon["Legendary"]==False]["Total"], equal_var = False)


Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957444e-47)

What do you conclude from this test? Write your conclusions below.

In [ ]:
# Your conclusions here:

# we can reject the hypothesis that the two groups are the same or similar even

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [8]:
# Your code here:

pokemon["Type 1"].value_counts()

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [9]:
# Your code here:

w_mean = pokemon[pokemon["Type 1"]=='Water']["Total"].mean()
mean = pokemon[pokemon["Type 1"]!='Water']["Total"].mean()
w_std = pokemon[pokemon["Type 1"]=='Water']["Total"].std(ddof=0)
std = pokemon[pokemon["Type 1"]!='Water']["Total"].std(ddof=0)
print(leg_mean)
print(nleg_mean)
print(leg_std)
print(nleg_std)

637.3846153846154
417.21360544217686
60.46682302213781
106.68776660321996


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [11]:
# Your code here:
st.ttest_ind(pokemon[pokemon["Type 1"]=='Water']["Total"],pokemon[pokemon["Type 1"]!='Water']["Total"], equal_var = True)



Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [ ]:
# Your conclusions here:

# The pvalue is around 65% so we cannot reject the hypothesis that the two groups are similar

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [13]:
# Your code here:

st.ttest_rel(pokemon["Defense"],pokemon["Attack"])


TtestResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

# pvalue is very low, well under 5% so we can refuse the hypothesis that the two dataframes are equal

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [14]:
# Your code here:

st.ttest_rel(pokemon["Sp. Atk"],pokemon["Sp. Def"])


TtestResult(statistic=0.853986188453353, pvalue=0.3933685997548122, df=799)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:

# pvalue is around 39%, well above 5% so we cannot reject the hypothesis that the the two groups are similar 
# they to not have a big difference

As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [15]:
# Your code here:
    
# H0: mu diff = 0
# H1: mu diff != 0 

st.ttest_1samp((pokemon["Defense"]-pokemon["Attack"]), 0)

# given we the p value we can refuse the hypothesis that the difference between defense and attach is anywhere near 0

TtestResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05, df=799)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

